In [ ]:
from dotenv import load_dotenv
from pydantic import BaseModel,Field
from typing import Annotated,TypedDict,List
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,START,END
import operator

In [ ]:
model=ChatGroq(model="llama-3.1-8b-instant")

In [ ]:
class EssaySchema(BaseModel):

    feedback:Annotated[str,Field(description="The feedback provided by the LLM")]
    score:Annotated[float,Field(description="The score given by the LLM",ge=0,le=10)]

In [ ]:
structured_model = model.with_structured_output(EssaySchema)

In [ ]:
essay='''The Nepal Premier League (NPL) is Nepal’s top professional T20 cricket league, launched to transform the country’s cricket scene and fan culture. It rose quickly from an idea to one of the most hyped sporting events in the country.

Origin and background
The NPL was officially established in 2024 as a franchise-based men’s T20 league organized by the Cricket Association of Nepal (CAN).​
Its creation followed the controversial Nepal T20 League and came at a time when the national team had just qualified for the 2024 Men’s T20 World Cup, which created huge optimism around Nepali cricket.​

From controversy to opportunity
Before the NPL, Nepal’s first big franchise attempt, the Nepal T20 League, had been damaged by allegations of match-fixing and corruption, which hurt public trust.​
The NPL was presented as a fresh start, with stricter oversight and a more professional structure, helping to rebuild credibility and attract both players and sponsors.​

Structure and teams
The league features eight city- or province-based franchises, such as Biratnagar Kings, Kathmandu Gorkhas, Pokhara Avengers, Lumbini Lions, and Janakpur Bolts.​
Teams are built through player auctions and include a mix of national stars, young Nepali talents, and well-known international cricketers, which raises both the standard of play and fan excitement.​

Why it got so hyped
Several factors drove the hype: Nepal’s rising cricket fandom, World Cup qualification, and the promise of a “homegrown IPL-style” league.​
Organisers deliberately branded the NPL as a “festival of the Himalayas,” combining high-quality cricket with music, entertainment, social media campaigns, and strong storytelling around local heroes.​

Role of media and social platforms
Television coverage, live streaming, and constant clips on TikTok, Facebook, and YouTube turned matches into national events rather than just sports fixtures.​
Fan discussions, memes, and support for regional teams online helped create rivalries and identities—Kathmandu vs Pokhara, Lumbini vs Sudurpaschim—that pulled even casual viewers into the league.​

Corporate money and professionalism
Big sponsors such as Siddhartha Bank (as title sponsor) and brands like Ncell, Dream11, and Red Bull poured money into the league, allowing better production, player payments, and facilities.​
Franchises together invested over NPR 160 million in teams and operations in the early seasons, signalling to fans that this was a serious, long-term project rather than a one-off tournament.​

Stadium atmosphere and fan experience
The TU International Cricket Ground in Kirtipur was upgraded with digital scoreboards, VIP areas, fan zones, and better seating, giving spectators an experience closer to international venues.​
Match days turned into festivals, with music, food stalls, and interactive zones, encouraging families and young people to attend and making the league a social event as much as a sporting one.​

Impact on players and youth
For Nepali players, the NPL created a new pathway to earn professional-level income and to share a dressing room with international stars, accelerating their growth and confidence.​
The league inspired many young fans to take cricket more seriously, seeing that a professional career was possible inside Nepal rather than only abroad.​

Economic and social ripple effects
The NPL boosted local businesses around match venues and increased demand for hotels, restaurants, and transport, especially in Kathmandu and tourist hubs like Pokhara.​
Travel packages around NPL matches as well as increased international interest in Nepal’s cricket story tied the league to tourism and broader economic hope in a sluggish economy.​

Recent seasons and growing legacy
The inaugural season in 2024 saw Janakpur Bolts win the first title, while by 2025 the league’s second edition, branded as Siddhartha Bank NPL 2025, featured 32 matches played in a packed November–December window.​
With teams like Lumbini Lions lifting the 2025 trophy and star-studded auctions attracting hundreds of players, the NPL has begun to establish traditions, rivalries, and records that deepen its place in Nepal’s sporting culture.​

This combination of timing, emotion, professionalism, and smart branding is what turned the Nepal Premier League from a new idea into a hyped reality that now shapes how Nepal watches and dreams about cricket.'''

In [ ]:
# Just a basic look at how we can use pydantic to allow us to give a structured output.
prompt=f"Provide a detailed feedback on the given essay.{essay}"
structured_model.invoke(prompt).score

In [ ]:
class EssayState(TypedDict):
    essay:str

    lang_feedback:str
    quality_feedback:str
    understanding_feedback:str

    final_feedback:str

    final_score:Annotated[List[float],operator.add]
    
    avg_score:float


In [ ]:
def language(state:EssayState):
    prompt=f"Provide a detailed feedback on the essay based on language and a score out of 10.{state['essay']}.Return: - feedback: detailed written feedback - score: a numeric score out of 10 (number only)"
    result=structured_model.invoke(prompt)

    return {'lang_feedback':result.feedback,'final_score':[result.score]}
    

In [ ]:
def quality(state:EssayState):
    prompt=f"Provide a detailed feedback on the essay based on quality and a score out of 10.\n{state['essay']}.Return: - feedback: detailed written feedback - score: a numeric score out of 10 (number only)"
    result=structured_model.invoke(prompt)

    return {'quality_feedback':result.feedback,'final_score':[result.score]}

In [ ]:
def understanding(state:EssayState):
    prompt=f"Provide a detailed feedback on the essay based on understanding and a score out of 10.\n{state['essay']}.Return: - feedback: detailed written feedback - score: a numeric score out of 10 (number only)"
    result=structured_model.invoke(prompt)

    return {'understanding_feedback':result.feedback,'final_score':[result.score]}

In [ ]:
def summary(state:EssayState):
    prompt=f"Based on the following feedback language:{state['lang_feedback']},\nquality:{state['quality_feedback']}\n,understanding:{state['understanding_feedback']}"
    result=model.invoke(prompt).content

    avg_score=sum(state["final_score"])/len(state["final_score"])

    return {'final_feedback': result, 'avg_score': avg_score}


In [ ]:
graph=StateGraph(EssayState)

In [ ]:
graph.add_node('language',language)
graph.add_node('quality',quality)
graph.add_node('understanding',understanding)
graph.add_node('summary',summary)

graph.add_edge(START,'language')
graph.add_edge(START,'quality')
graph.add_edge(START,'understanding')

graph.add_edge('language','summary')
graph.add_edge('quality','summary')
graph.add_edge('understanding','summary')

graph.add_edge('summary',END)

In [ ]:
workflow=graph.compile()
workflow

In [ ]:
intial_state = {
    'essay': essay
}

workflow.invoke(intial_state)